In [1]:
%%capture
%pip install - r requirement.txt
# python==3.6.13

In [2]:
import numpy as np
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMaximize, LpMinimize, LpBinary, LpStatus, value, PULP_CBC_CMD
import pulp as pl
import time
import heapq
import json
import psycopg2
import psycopg2.extras

debug = 1  # debug mode
TIMEOUT = 500 # timeout 
if debug:
    pd.set_option('display.max_columns', 500)  # show all columns

numLimit = 5
sql_command1 = "select billing_status_fz as billing, unit_id_fz, product, fleet_year_fz as fleet_year, contract_cust_id as customer, contract_lease_type as contract, cost, nbv, age_x_ceu as weighted_age from ads.ads_bo_asset_init" 

n = 100000 # number of containers

### INPUT

#### Parameters

TODO: load from database and POST

In [3]:
with open("./parameterDemo.json") as f:
    paramDict = json.load(f)

In [4]:
queryID = paramDict['query_id']
initialQuery = paramDict['initial_query']
# TODO: link to database system
param = paramDict['secondary_query']
NbvCost = param['prefer']['NBVOrCost']
maxOrMin = param['prefer']['maxOrMin']

fleetAgeLowBound = [None for _ in range(numLimit)]
fleetAgeUpBound = [None for _ in range(numLimit)]
fleetAgeLimit = [None for _ in range(numLimit)]
fleetAgeGeq = [None for _ in range(numLimit)]
weightedAgeLowBound = [None for _ in range(numLimit)]
weightedAgeUpBound = [None for _ in range(numLimit)]
weightedAgeLimit = [None for _ in range(numLimit)]
weightedAgeGeq = [None for _ in range(numLimit)]
lesseeType = [None for _ in range(numLimit)]
lesseeLimit = [None for _ in range(numLimit)]
lesseeGeq = [None for _ in range(numLimit)]
productType = [None for _ in range(numLimit)]
productLimit = [None for _ in range(numLimit)]
productGeq = [None for _ in range(numLimit)]
contractType = [None for _ in range(numLimit)]
contractLimit = [None for _ in range(numLimit)]
contractGeq = [None for _ in range(numLimit)]

minTotalNbv = param['totalNBVFrom'] if param['totalNBVFrom'] else None
maxTotalNbv = param['totalNBVTo'] if param['totalNBVTo'] else None

minTotalCost = param['totalCostFrom'] if param['totalCostFrom'] else None
maxTotalCost = param['totalCostTo'] if param['totalCostTo'] else None

fleetAgeAvgLimit = param['containersAge']['average']['averageWeighedAge'] if param['containersAge']['average']['averageWeighedAge'] else None
fleetAgeAvgGeq = param['containersAge']['average']['symbol']
for i in range(len(param['containersAge']['list'])):
    fleetAgeLowBound[i] = param['containersAge']['list'][i]['containersAgeFrom']
    fleetAgeUpBound[i] = param['containersAge']['list'][i]['containersAgeTo']
    fleetAgeLimit[i] = param['containersAge']['list'][i]['%'] / 100
    fleetAgeGeq[i] = param['containersAge']['list'][i]['symbol']

weightedAgeAvgLimit = param['weightedAge']['average']['averageWeighedAge'] if param['weightedAge']['average']['averageWeighedAge'] else None
weightedAgeAvgGeq = param['weightedAge']['average']['symbol']
for i in range(len(param['weightedAge']['list'])):
    weightedAgeLowBound[i] = param['weightedAge']['list'][i]['weightedAgeFrom']
    weightedAgeUpBound[i] = param['weightedAge']['list'][i]['weightedAgeTo']
    weightedAgeLimit[i] = param['weightedAge']['list'][i]['%'] / 100
    weightedAgeGeq[i] = param['weightedAge']['list'][i]['symbol']

topLesseeLimit = [param['TopLessee'][0]['%'] / 100 if param['TopLessee'][0]['Top1'] else None,
               param['TopLessee'][1]['%'] / 100 if param['TopLessee'][1]['Top2'] else None,
               param['TopLessee'][2]['%'] / 100 if param['TopLessee'][2]['Top3'] else None]
topLesseeGeq = [param['TopLessee'][0]['symbol'] if param['TopLessee'][0]['Top1'] else None,
             param['TopLessee'][1]['symbol'] if param['TopLessee'][1]['Top2'] else None,
             param['TopLessee'][2]['symbol'] if param['TopLessee'][2]['Top3'] else None]
for i in range(len(param['lessee'])):
    lesseeType[i] = param['lessee'][i]['lessee']
    lesseeLimit[i] = param['lessee'][i]['%'] / 100
    lesseeGeq[i] = param['lessee'][i]['symbol']

OnHireLimit = param['status'][0]['%'] / 100 if param['status'][0]['onhire'] else None
OffHireLimit = param['status'][1]['%'] / 100 if param['status'][1]['offhire'] else None
NoneHireLimit = param['status'][2]['%'] / 100 if param['status'][2]['none'] else None

for i in range(len(param['product'])):
    productType[i] = param['product'][i]['productType']
    productLimit[i] = param['product'][i]['%'] / 100
    productGeq[i] = param['product'][i]['symbol']

for i in range(len(param['contractType'])):
    contractType[i] = param['contractType'][i]['contractType']
    contractLimit[i] = param['contractType'][i]['%'] / 100
    contractGeq[i] = param['contractType'][i]['symbol']

#### Load Data

In [10]:
try:
    conn = psycopg2.connect(host = "10.18.35.245", port = "5432", dbname = "iflorensgp_gy", user = "flgyads", password = "flgyads!0920")
    rawdata = pd.read_sql(sql_command1, conn)[:n]
    conn.close()
except:
    print("Load data from GreenPlum failure !")

In [ ]:
# rawData = pd.read_excel(io='./20220630_Overall_Fleet_(FIR)_FCI_combined(with planned)_iflorens_simplified.xlsb', sheet_name='Raw', engine='pyxlsb')
# data = rawData.iloc[:n].reset_index()
# data.drop('index', axis=1, inplace=True)
data = rawData[:n]
data.head(3)

NameError: name 'rawData' is not defined

### DATA PREPARATION

#### Container Age

Assign to `FleetAge{0}.format(i)`

In [84]:
for i in range(5):
    if fleetAgeLimit[i]:
        column_name = 'FleetAge{0}'.format(i)
        data[column_name] = data['fleet_year'].apply(lambda x: 1 if fleetAgeLowBound[i]<=x<fleetAgeUpBound[i] else 0)

FleetAge0
FleetAge1


#### Status

Assign new column `Status`

In [85]:
data['OnHireStatus'] = data['billing'].apply(lambda x: 1 if x=='ON' else 0)
data['OffHireStatus'] = data['billing'].apply(lambda x: 1 if x=='OF' else 0)
data['NoneStatus'] = data['billing'].apply(lambda x: 1 if pd.isna(x) else 0)

#### Weighted Age

Assign new column `WeightedAge{0}.format(i)`

In [ ]:
for i in range(5):
    if weightedAgeLimit[i]:
        column_name = 'WeightedAge{0}'.format(i)
        data[column_name] = data['weighted_age'].apply(lambda x: 1 if weightedAgeLowBound[i]<=x<weightedAgeUpBound[i] else 0)


WeightedAge0
WeightedAge1


#### Product

Assign new column `ProductType{0}.format(i)`

In [87]:
for i in range(5):
    if productLimit[i]:
        column_name = 'ProductType{0}'.format(i)
        data[column_name] = data['product'].apply(lambda x: 1 if x in productType[i] else 0)

ProductType0
ProductType1


#### Lessee

Assign new column `Lessee{0}.format(i)`

In [88]:
# ONE HOT -- all lessee
for lesseeName in data['customer'].value_counts().index:
    data[lesseeName] = data['customer'].apply(lambda x: 1 if x==lesseeName else 0)


#### Contract Type

Assign new column `ContractType{0}.format(i)`

In [89]:
for i in range(5):
    if contractLimit[i]:
        column_name = 'ContractType{0}'.format(i)
        data[column_name] = data['contract'].apply(lambda x: 1 if x in contractType[i] else 0)

ContractType0
ContractType1


### SAVE DATA

In [90]:
if debug:
    data.to_csv('prepared_data_demo.csv')
    data = pd.read_csv('./prepared_data_demo.csv')
data.head(3)

,Unnamed: 0,Unit Id Fz,Cost,Product,Contract Cust Id,Contract Lease Type,Nbv,Billing Status Fz,Fleet Year Fz,Age x CEU,FleetAge0,FleetAge1,OnHireStatus,OffHireStatus,NoneStatus,WeightedAge0,WeightedAge1,ProductType0,ProductType1,MSC,ContractType0,ContractType1
0,0,MSDU2865003,3690.0,D20,MSC,LP,3635.841,ON,0.52,0.52,1,0,1,0,0,0,0,0,1,1,0,1
1,1,MSDU2865019,3690.0,D20,MSC,LP,3635.841,ON,0.52,0.52,1,0,1,0,0,0,0,0,1,1,0,1
2,2,MSDU2865030,3690.0,D20,MSC,LP,3635.841,ON,0.52,0.52,1,0,1,0,0,0,0,0,1,1,0,1


### Model Part

#### Prepare Parameters

convert to numpy

In [91]:
nbv = data['nbv'].to_numpy()
cost = data['cost'].to_numpy()
onHireStatus = data['OnHireStatus'].to_numpy()
offHireStatus = data['OffHireStatus'].to_numpy()
noneHireStatus = data['NoneStatus'].to_numpy()

fleetAge = []
weightedAge = []
product = []
contract = []
for i in range(numLimit):
    fleetAge.append(data['FleetAge{0}'.format(i)].to_numpy() if fleetAgeLimit[i] else None)
    weightedAge.append(data['WeightedAge{0}'.format(i)].to_numpy() if weightedAgeLimit[i] else None)
    product.append(data['ProductType{0}'.format(i)].to_numpy() if productLimit[i] else None)
    contract.append(data['ContractType{0}'.format(i)].to_numpy() if contractLimit[i] else None)

fleetAgeAvg = data['fleet_year'].to_numpy()
weightedAgeAvg = data['weighted_age'].to_numpy()

lesseeOneHot = {lesseeName: data[lesseeName].to_numpy() for lesseeName in data['customer'].value_counts().index}


if debug:
    print('nbv:', nbv.shape)
    print('cost:', cost.shape)
    print('onHireStatus:', onHireStatus.shape)
    print('offHireStatus:', offHireStatus.shape)
    print('noneHireStatus:', noneHireStatus.shape)
    print('fleetAgeAvg:', fleetAgeAvg.shape)
    print('weightedAgeAvg:', weightedAgeAvg.shape)

    print('fleetAge:', fleetAge)
    print('weightedAge:', weightedAge)
    print('productType:', product)
    print('contractType:', contract)

nbv: (100000,)
cost: (100000,)
onHireStatus: (100000,)
offHireStatus: (100000,)
noneHireStatus: (100000,)
fleetAgeAvg: (100000,)
weightedAgeAvg: (100000,)
fleetAge: [array([1, 1, 1, ..., 1, 1, 1]), array([0, 0, 0, ..., 0, 0, 0]), None, None, None]
weightedAge: [array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]), None, None, None]
productType: [array([0, 0, 0, ..., 1, 1, 1]), array([1, 1, 1, ..., 0, 0, 0]), None, None, None]
contractType: [array([0, 0, 0, ..., 1, 1, 1]), array([1, 1, 1, ..., 0, 0, 0]), None, None, None]


#### Constraints

#### Objective

1. min Cost: `Vars @ cost`

2. max Nbv: `Vars @ nbv`

#### Define Problem

In [108]:
def SortTop(l, n):
    topN = heapq.nlargest(n, l, key=lambda x:x[1])
    return np.sum(np.stack([lesseeOneHot[topN[i][0]] for i in range(n)]), axis=0)


In [109]:
var = np.array([LpVariable('container_{0}'.format(i), lowBound=0, cat=LpBinary) for i in range(nbv.shape[0])])
prob = LpProblem("MyProblem", LpMaximize if maxOrMin else LpMinimize)

TODO: remove warm up part, fix with warm start.

In [110]:
warmProb = LpProblem("WarmProblem", LpMaximize)
warmProb += lpSum(var * 1)
warmProb.solve(PULP_CBC_CMD(msg = False, timeLimit=1))

1

In [111]:
# objective function 
if NbvCost:
    prob += lpSum(var * nbv)
else:
    prob += lpSum(var * cost)

In [112]:
# constraints
numSelected = lpSum(var) # num of selected containers

# nbv
if maxTotalNbv:
    prob += lpSum(var * nbv) <= maxTotalNbv
if minTotalNbv:
    prob += lpSum(var * nbv) >= minTotalNbv
    
# cost
if maxTotalCost:
    prob += lpSum(var * cost) <= maxTotalCost
if minTotalCost:
    prob += lpSum(var * cost) >= minTotalCost

# status
if OnHireLimit:
    prob += lpSum(var * onHireStatus) >= OnHireLimit * numSelected
if OffHireLimit:
    prob += lpSum(var * offHireStatus) <= OffHireLimit * numSelected
if NoneHireLimit:
    prob += lpSum(var * noneHireStatus) <= NoneHireLimit * numSelected
  
# container age
if fleetAgeAvgLimit:
    if fleetAgeAvgGeq:
        prob += lpSum(var * fleetAgeAvg) >= fleetAgeAvgLimit * numSelected
    else:
        prob += lpSum(var * fleetAgeAvg) <= fleetAgeAvgLimit * numSelected
for i in range(numLimit):
    if fleetAgeLimit[i]:
        if fleetAgeGeq[i]:
            prob += lpSum(var * fleetAge[i]) >= fleetAgeLimit[i] * numSelected
        else:
            prob += lpSum(var * fleetAge[i]) <= fleetAgeLimit[i] * numSelected

# weighted age
if weightedAgeAvgLimit:
    if weightedAgeAvgGeq:
        prob += lpSum(var * weightedAgeAvg) >= weightedAgeAvgLimit * numSelected
    else:
        prob += lpSum(var * weightedAgeAvg) <= weightedAgeAvgLimit * numSelected
for i in range(numLimit):
    if weightedAgeLimit[i]:
        if weightedAgeGeq[i]:
            prob += lpSum(var * weightedAge[i]) >= weightedAgeLimit[i] * numSelected
        else:
            prob += lpSum(var * weightedAge[i]) <= weightedAgeLimit[i] * numSelected

# product
for i in range(numLimit):
    if productLimit[i]:
        if productGeq[i]:
            prob += lpSum(var * product[i]) >= productLimit[i] * numSelected
        else:
            prob += lpSum(var * product[i]) <= productLimit[i] * numSelected

# lessee
for i in range(numLimit):
    if lesseeLimit[i] and lesseeType[i] in lesseeOneHot:
        if lesseeGeq[i]:
            prob += lpSum(var * lesseeOneHot[lesseeType[i]]) >= lesseeLimit[i] * numSelected
        else:
            prob += lpSum(var * lesseeOneHot[lesseeType[i]]) <= lesseeLimit[i] * numSelected


# find top1, top2, top3
for i in range(min(3, len(lesseeOneHot))):
    if topLesseeLimit[i]:
        if topLesseeGeq[i]:
            prob += lpSum(var * SortTop(list({i: value(lpSum(var * lesseeOneHot[i])) for i in lesseeOneHot}.items()), i+1)) >= topLesseeLimit[i] * numSelected
        else:
            prob += lpSum(var * SortTop(list({i: value(lpSum(var * lesseeOneHot[i])) for i in lesseeOneHot}.items()), i+1)) <= topLesseeLimit[i] * numSelected

# contract type
for i in range(numLimit):
    if contractLimit[i]:
        if contractGeq[i]:
            prob += lpSum(var * contract[i]) >= contractLimit[i] * numSelected
        else:
            prob += lpSum(var * contract[i]) <= contractLimit[i] * numSelected


[('MSC', 100000.0)]
[('MSC', 100000.0)]


In [38]:
solver = PULP_CBC_CMD(msg = True, timeLimit=TIMEOUT)
prob.solve(solver)

1

In [39]:
print("==============================================================")
# print(prob)
print("status:", LpStatus[prob.status])
print("==============================================================")
print("target value: ", value(prob.objective))

status: Optimal
target value:  188855358.6410332


In [23]:
# if solution is found
if debug:
    result = np.array([var[i].varValue for i in range(n)])
    print(int(sum(result)), '/', n, 'containers are selected.')

61837 / 200000 containers are selected.


### Analysis

In [41]:
if debug:    
    result = np.array([var[i].varValue for i in range(n)])
    print(int(sum(result)), '/', n, 'containers are selected.')
    print('======================================================================')
    print("nbv: {0} between {1} - {2}".format(round(sum(result * nbv), 2), minTotalNbv, maxTotalNbv))
    print("cost: {0} between {1} - {2}".format(round(sum(result * cost), 2), minTotalCost, maxTotalCost))
    print('billing status:')
    if OnHireLimit:
        print('\t onhire {0}, -- {1}'.format(sum(result * onHireStatus)/sum(result), OnHireLimit))
    if OffHireLimit:
        print('\t offhire {0}, -- {1}'.format(sum(result * offHireStatus)/sum(result), OffHireLimit))
    if NoneHireLimit:
        print('\t none {0}, -- {1}'.format(sum(result * noneHireStatus)/sum(result), NoneHireLimit))

    print("container age:")
    print('\t container average age is {0}, -- {1}'.format(round(sum(result * fleetAgeAvg)/sum(result), 2), fleetAgeAvgLimit))
    for i in range(numLimit):
        if fleetAgeLimit[i]  :
            print("\t container age from {0} to {1} is {2}, -- {3}:".format(fleetAgeLowBound[i], fleetAgeUpBound[i], round(sum(result * fleetAge[i])/sum(result), 2), fleetAgeLimit[i]))

    print("weighted age:")
    print('\t weighted average age is {0}, -- {1}'.format(round(sum(result * weightedAgeAvg)/sum(result), 2), weightedAgeAvgLimit))
    for i in range(numLimit):
        if weightedAgeLimit[i]  :
            print("\t weighted age from {0} to {1} is {2}, -- {3}:".format(weightedAgeLowBound[i], weightedAgeUpBound[i], round(sum(result * weightedAge[i])/sum(result), 2), weightedAgeLimit[i]))    

    print("product:")
    for i in range(numLimit):
        if productLimit[i]  :
            print("\t product {0} is {1}, -- {2}:".format(productType[i], round(sum(result * product[i])/sum(result), 2), productLimit[i]))    
    
    print("lessee:")
    for i in range(numLimit):
        if lesseeLimit[i]  :
            print("\t lessee {0} is {1}, -- {2}:".format(lesseeType[i], round(sum(result * lesseeOneHot[lesseeType[i]])/sum(result), 2), lesseeLimit[i]))    

    print('Top lessee:')
    numLessee = {lesseeName: value(lpSum(var * lesseeOneHot[lesseeName])) for lesseeName in data['Contract Cust Id'].value_counts().index}
    sortedLessee = list(numLessee.items())
    top3Lessee = heapq.nlargest(3, sortedLessee, key=lambda x:x[1])
    if topLesseeLimit[0]:
        print('\t top 1 {0} is {1}, -- {2}'.format(top3Lessee[0][0], top3Lessee[0][1] / sum(result), topLesseeLimit[0]))
    if topLesseeLimit[1]:
        print('\t top 2 {0} {1} is {2}, -- {3}'.format(top3Lessee[0][0], top3Lessee[1][0], (top3Lessee[0][1] + top3Lessee[1][1])/ sum(result), topLesseeLimit[1]))
    if topLesseeLimit[2]:
        print('\t top 3 {0} {1} {2} is {3}, -- {4}'.format(top3Lessee[0][0], top3Lessee[1][0], top3Lessee[2][0], (top3Lessee[0][1] + top3Lessee[1][1] + top3Lessee[2][1])/ sum(result), topLesseeLimit[2]))

    print("contract type:")
    for i in range(numLimit):
        if contractLimit[i]  :
            print("\t contract type {0} is {1}, -- {2}:".format(contractType[i], round(sum(result * contract[i])/sum(result), 2), contractLimit[i])) 


36800 / 100000 containers are selected.
nbv: 188855358.64 between 100000000 - 200000000
cost: 199999996.68 between 100000000 - 200000000
billing status:
	 offhire 0.010625, -- 0.15
container age:
	 container average age is 1.55, -- 3
	 container age from 0 to 3 is 0.94, -- 0.7:
	 container age from 3 to 6 is 0.01, -- 0.01:
weighted age:
	 weighted average age is 3.48, -- 7
	 weighted age from 3 to 6 is 0.15, -- 0.15:
	 weighted age from 4 to 15 is 0.14, -- 0.01:
product:
	 product ['D4H'] is 0.69, -- 0.5:
	 product ['D20', 'D40'] is 0.22, -- 0.1:
lessee:
	 lessee MSC is 0.49, -- 0.6:
	 lessee ESSC is 0.05, -- 0.2:
Top lessee:
	 top 1 MSC is 0.48733695652173914, -- 0.6
	 top 2 MSC ESSC is 0.5331521739130435, -- 0.8
contract type:
	 contract type ['LT'] is 0.68, -- 0.6:
	 contract type ['LP', 'LM'] is 0.1, -- 0.1:


### Output

In [25]:
if prob.status == 1 or prob.status == 2:
    data.insert(loc=0, column="Selected", value=result)
    data = data[['Unit Id Fz', 'Cost', 'Product', 'Contract Cust Id', 'Contract Lease Type', 'Nbv', 'Billing Status Fz', 'Fleet Year Fz', 'Age x CEU']]
    data.to_csv('demo_result.csv')